# Example data anonymization

In Pega CDH 8.5 and up, it's now possible to record the historical data as seen by the Adaptive Models. See [this academy challenge](https://academy.pega.com/challenge/exporting-historical-data/v4) for reference. This historical data can be further used to experiment with offline models, but also to fine-tune the OOTB Gradient Boosting model. However, sharing this information with Pega can be sensitive as it contains raw predictor data. 

To this end, we provide a simple and transparent script to fully anonimize this dataset.

The DataAnonymization script is now part of pdstools, and you can import it directly as such.

In [1]:
# These lines are only for rendering in the docs, and are hidden through Jupyter tags
# Do not run if you're running the notebook seperately

import os  
import sys
import plotly.io as pio
pio.renderers.default = "notebook_connected"

sys.path.append("../../../")
sys.path.append('../../python')

In [2]:
from pdstools import ADMDatamart
from pdstools import Config, DataAnonymization
import polars as pl

## Input data

To demonstrate this process, we're going to anonymise this toy example dataframe:

In [3]:
pl.read_ndjson('../../../../data/SampleHDS.json')

Context_Name,Customer_MaritalStatus,Customer_CLV,Customer_City,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,i64,str,str,str
"""FirstMortgage3…","""Married""",1460,"""Port Raoul""","""Account""","""Rejected"""
"""FirstMortgage3…","""Unknown""",669,"""Laurianneshire…","""AutoLoans""","""Accepted"""
"""MoneyMarketSav…","""No Resp+""",1174,"""Jacobshaven""","""Account""","""Rejected"""
"""BasicChecking""","""Unknown""",1476,"""Lindton""","""Account""","""Rejected"""
"""BasicChecking""","""Married""",1211,"""South Jimmiesh…","""DepositAccount…","""Accepted"""
"""UPlusFinPerson…","""No Resp+""",533,"""Bergeville""",null,"""Rejected"""
"""BasicChecking""","""No Resp+""",555,"""Willyville""","""DepositAccount…","""Rejected"""


As you can see, this dataset consists of regular predictors, IH predictors, context keys and the outcome column. Additionally, some columns are numeric, others are strings. Let's first initialize the DataAnonymization class.

In [4]:
anon = DataAnonymization(hds_folder='../../../../data/')

By default, the class applies a set of anonymisation techniques:
- Column names are remapped to a non-descriptive name
- Categorical values are hashed with a random seed
- Numerical values are normalized between 0 and 1
- Outcomes are mapped to a binary outcome.

To apply these techniques, simply call `.process()`:

In [5]:
anon.process()

PREDICTOR_0,PREDICTOR_1,PREDICTOR_2,filename,Context_Name,IH_PREDICTOR_0,Decision_Outcome
f64,str,str,str,str,str,bool
1.2927e19,"""82788226952000…","""14404453783465…","""../../../../da…","""14598137818554…","""91781133765226…",false
1.4856e19,"""14787566814202…","""92228001274259…","""../../../../da…","""14598137818554…","""57991994733241…",true
5.6458e17,"""12474199030582…","""64715022596660…","""../../../../da…","""17147609693198…","""91781133765226…",false
4.0723e18,"""14787566814202…","""74403298913235…","""../../../../da…","""12655594822103…","""91781133765226…",false
1.4677e19,"""82788226952000…","""13921662552763…","""../../../../da…","""12655594822103…","""93561984507846…",true
1.6330e19,"""12474199030582…","""10284470011504…","""../../../../da…","""11854492028158…",null,false
8.7675e18,"""12474199030582…","""14272679294982…","""../../../../da…","""12655594822103…","""93561984507846…",false


To trace back the columns to their original names, the class also contains a mapping, which does not have to be provided.

In [6]:
anon.column_mapping

{'Customer_CLV': 'PREDICTOR_0',
 'Customer_MaritalStatus': 'PREDICTOR_1',
 'Customer_City': 'PREDICTOR_2',
 'filename': 'filename',
 'Context_Name': 'Context_Name',
 'IH_Web_Inbound_Accepted_pxLastGroupID': 'IH_PREDICTOR_0',
 'Decision_Outcome': 'Decision_Outcome'}

## Configs

Each capability can optionally be turned off - see below for the full list of config options, and refer to the API reference for the full description.

In [7]:
dict(zip(Config.__init__.__code__.co_varnames[1:], Config.__init__.__defaults__))

{'config_file': None,
 'hds_folder': '.',
 'use_datamart': False,
 'datamart_folder': 'datamart',
 'output_format': 'ndjson',
 'output_folder': 'output',
 'mapping_file': 'mapping.map',
 'mask_predictor_names': True,
 'mask_context_key_names': False,
 'mask_ih_names': True,
 'mask_outcome_name': False,
 'mask_predictor_values': True,
 'mask_context_key_values': True,
 'mask_ih_values': True,
 'mask_outcome_values': True,
 'context_key_label': 'Context_*',
 'ih_label': 'IH_*',
 'outcome_column': 'Decision_Outcome',
 'positive_outcomes': ['Accepted', 'Clicked'],
 'negative_outcomes': ['Rejected', 'Impression'],
 'special_predictors': ['Decision_DecisionTime',
  'Decision_OutcomeTime',
  'Decision_Rank'],
 'sample_percentage_schema_inferencing': 0.01}

It's easy to change these parameters by just passing the keyword arguments. In the following example, we
- Keep the IH predictor names
- Keep the outcome values
- Keep the context key values
- Keep the context key predictor names

In [8]:
anon = DataAnonymization(
    hds_folder="../../../../data/",
    mask_ih_names=False,
    mask_outcome_values=False,
    mask_context_key_values=False,
    mask_context_key_names=False,
)
anon.process()


PREDICTOR_0,PREDICTOR_1,PREDICTOR_2,filename,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
f64,str,str,str,str,str,str
1.2927e19,"""15437881945743…","""12580457169219…","""../../../../da…","""FirstMortgage3…","""17035321243206…","""Rejected"""
1.4856e19,"""12357540970178…","""46654266495786…","""../../../../da…","""FirstMortgage3…","""16416612856217…","""Accepted"""
5.6458e17,"""36654279770324…","""15867674402710…","""../../../../da…","""MoneyMarketSav…","""17035321243206…","""Rejected"""
4.0723e18,"""12357540970178…","""20030455846574…","""../../../../da…","""BasicChecking""","""17035321243206…","""Rejected"""
1.4677e19,"""15437881945743…","""14084368269330…","""../../../../da…","""BasicChecking""","""11572403226022…","""Accepted"""
1.6330e19,"""36654279770324…","""54633815000530…","""../../../../da…","""UPlusFinPerson…",null,"""Rejected"""
8.7675e18,"""36654279770324…","""14651249195483…","""../../../../da…","""BasicChecking""","""11572403226022…","""Rejected"""


The configs can also be written and read as such:

In [9]:
anon.config.save_to_config_file('config.json')

In [10]:
anon = DataAnonymization(config=Config(config_file='config.json'))
anon.process()

PREDICTOR_0,PREDICTOR_1,PREDICTOR_2,filename,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
f64,str,str,str,str,str,str
1.2927e19,"""31991989774893…","""10078383889993…","""../../../../da…","""FirstMortgage3…","""16486022770495…","""Rejected"""
1.4856e19,"""10268298185647…","""10391394201979…","""../../../../da…","""FirstMortgage3…","""30606002633453…","""Accepted"""
5.6458e17,"""15244227784035…","""91005488773254…","""../../../../da…","""MoneyMarketSav…","""16486022770495…","""Rejected"""
4.0723e18,"""10268298185647…","""37893269658769…","""../../../../da…","""BasicChecking""","""16486022770495…","""Rejected"""
1.4677e19,"""31991989774893…","""16016399456350…","""../../../../da…","""BasicChecking""","""15581595688072…","""Accepted"""
1.6330e19,"""15244227784035…","""11549072889594…","""../../../../da…","""UPlusFinPerson…",null,"""Rejected"""
8.7675e18,"""15244227784035…","""71438206744894…","""../../../../da…","""BasicChecking""","""15581595688072…","""Rejected"""


## Exporting
Two functions export:
- `create_mapping_file()` writes the mapping file of the predictor names
- `write_to_output()` writes the processed dataframe to disk

Write to output accepts the following extensions: `["ndjson", "parquet", "arrow", "csv"]`

In [11]:
anon.create_mapping_file()
with open('mapping.map') as f:
    print(f.read())

Customer_CLV=PREDICTOR_0
Customer_MaritalStatus=PREDICTOR_1
Customer_City=PREDICTOR_2
filename=filename
Context_Name=Context_Name
IH_Web_Inbound_Accepted_pxLastGroupID=IH_Web_Inbound_Accepted_pxLastGroupID
Decision_Outcome=Decision_Outcome



In [12]:
anon.write_to_output(ext='arrow')

In [13]:
pl.read_ipc('output/hds.arrow')

PREDICTOR_0,PREDICTOR_1,PREDICTOR_2,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
f64,str,str,str,str,str
1.2927e19,"""15265702868687…","""84144520360952…","""FirstMortgage3…","""16128778087624…","""Rejected"""
1.4856e19,"""14301223080907…","""34458363148480…","""FirstMortgage3…","""78458246873203…","""Accepted"""
5.6458e17,"""65996278119921…","""27154055810724…","""MoneyMarketSav…","""16128778087624…","""Rejected"""
4.0723e18,"""14301223080907…","""14550705710554…","""BasicChecking""","""16128778087624…","""Rejected"""
1.4677e19,"""15265702868687…","""17927858130314…","""BasicChecking""","""12570385444763…","""Accepted"""
1.6330e19,"""65996278119921…","""43196528413729…","""UPlusFinPerson…",null,"""Rejected"""
8.7675e18,"""65996278119921…","""26652838409653…","""BasicChecking""","""12570385444763…","""Rejected"""


## Advanced: Hash fuctions

By default, we use [the same hashing algorithm Polars](https://pola-rs.github.io/polars/py-polars/html/reference/expressions/api/polars.Expr.hash.html#polars.Expr.hash) uses: [xxhash](https://github.com/Cyan4973/xxHash), as implemented [here](https://github.com/pola-rs/polars/blob/3f287f370b3c388ed2f3f218b2c096382548136f/polars/polars-core/src/vector_hasher.rs#L266). xxhash is fast to compute, and you can check its performance in collision, dispersion and randomness [here](https://github.com/Cyan4973/xxHash/tree/dev/tests). 

xxhash accepts four distinct seeds, but by default we set the seeds to `0`. It is possible to set the `seed` argument of the `process()` function to `'random'`, which will set all four seeds to a random integer between `0` and `1000000000`. Alternatively, it is possible to supply the four seeds manually with arguments `seed`, `seed_1`, `seed_2` and `seed_3`. 

If the xxhash with (random) seed(s) is not deemed sufficiently secure, it is possible to use your own hashing algorithm.

Note that since we're now running python code and not native Polars code anymore, this will be _significantly_ slower. Nonetheless, it is possible.

Just as an example - this is how one would use sha3_256:

In [14]:
from hashlib import sha3_256

anon.process(algorithm=lambda x: sha3_256(x.encode()).hexdigest())

ComputeError: AttributeError: 'int' object has no attribute 'encode'